In [1]:
!pip install -q duckdb
!pip install -q deltalake

StatementMeta(, f44a4e91-0fab-4ba0-9fea-211c188fcff7, 3, Finished, Available)

In [2]:
#the function read all the tables in the default Lakehouse and save it in a Table Delta_stats, change the url to reflect your destination
import duckdb 
import pandas as pd
from deltalake import DeltaTable
import os
from datetime import datetime
from deltalake.writer import write_deltalake
def get_stats(Source,Destination,mode):
    appended_data = []
    List_tables = [ f.name for f in os.scandir(Source) if f.is_dir() ]
    stat_table = os.path.basename(os.path.normpath(Destination))
    if stat_table in List_tables: List_tables.remove(stat_table)
    for tbl in List_tables :
        dt = DeltaTable(Source+ tbl)
        xx= dt.get_add_actions(flatten=True).to_pandas()
        if 'tags.VORDER' in xx.columns:
             vorder =True
        else :
             vorder =False
        deltalog=pd.DataFrame([[tbl,xx['size_bytes'].sum(),vorder]],columns=['tbl','size_bytes','vorder'])
        delta = [Source+tbl + "/" + i for i in dt.files()]
        duckdb.sql(f'from parquet_scan({delta})').to_view(tbl)
        df= duckdb.sql(f'''
        with x as (
        select file_name,num_rows,num_row_groups, '{tbl}' as tbl from parquet_file_metadata({delta})
        )
        select x.tbl, file_name,num_rows,num_row_groups, ceil(size_bytes/(1024*1024)) as size,vorder from x
        left join deltalog as y
        on x.tbl=y.tbl
         ''').df()
        appended_data.append(df)
    appended_data = pd.concat(appended_data,ignore_index=True)
    appended_data['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    write_deltalake(Destination,appended_data,mode=mode,storage_options={"allow_unsafe_rename":"true"})

    dt = DeltaTable(Destination)
    delta = [Destination + "/" + i for i in dt.files()]
    duckdb.sql(f'from parquet_scan({delta})').to_view("xxx")

    duckdb.sql('''
    select tbl,
    sum(num_rows) as total_rows,
    count(*) as num_files,
    sum(num_row_groups) as num_row_group,
    cast(ceil(total_rows/num_row_group) as integer) as Average_row_group,
    min(size) as file_size_MB,
    vorder
    from xxx 
    where timestamp= ( select max(timestamp) from xxx)
    group by all
    order by total_rows desc
    ''').show()

StatementMeta(, f44a4e91-0fab-4ba0-9fea-211c188fcff7, 4, Finished, Available)

In [3]:
### to keep History use append , overwrite will keep only the latest data
get_stats('/lakehouse/default/Tables/','/lakehouse/default/Files/xx_deltastats',"overwrite")

StatementMeta(, f44a4e91-0fab-4ba0-9fea-211c188fcff7, 5, Finished, Available)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┬────────────┬───────────┬───────────────┬───────────────────┬──────────────┬─────────┐
│     tbl      │ total_rows │ num_files │ num_row_group │ Average_row_group │ file_size_MB │ vorder  │
│   varchar    │   int128   │   int64   │    int128     │       int32       │    double    │ boolean │
├──────────────┼────────────┼───────────┼───────────────┼───────────────────┼──────────────┼─────────┤
│ scada_python │  230363649 │        30 │           235 │            980271 │       2051.0 │ false   │
│ scada_spark  │  230363649 │         7 │            35 │           6581819 │       1565.0 │ true    │
│ calendar     │       2467 │         1 │             1 │              2467 │          1.0 │ true    │
│ duid         │        606 │         1 │             1 │               606 │          1.0 │ true    │
└──────────────┴────────────┴───────────┴───────────────┴───────────────────┴──────────────┴─────────┘



StatementMeta(, f44a4e91-0fab-4ba0-9fea-211c188fcff7, 6, Finished, Available)